In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/home/shaoz/projects/collaborate/spgc/AD/MADReSS-23-train/training-groundtruth.csv')

In [3]:
df.head(5)

,adressfname,age,gender,educ,dx,mmse
0,adrso002,70,female,NaN,Control,26.0
1,adrso003,72,female,NaN,Control,30.0
2,adrso004,74,female,NaN,Control,30.0
3,adrso005,67,female,NaN,Control,27.0
4,adrso006,65,female,NaN,Control,28.0


In [4]:
df.shape

(237, 6)

In [10]:
list(df['dx'].unique())

['Control', 'ProbableAD']

In [7]:
item = df.loc[2,:]

In [8]:
item['dx']

'Control'